## Hierarchical Agent Graph for Multi Agent Collaboration



In this section, we will explore the different types of multi-agent systems you can build with **Strands Agents**. Multi-agent systems consist of multiple interacting intelligent agents that enable:

- **Distributed Problem Solving**: Breaking complex tasks into subtasks for parallel processing  
- **Information Sharing**: Agents exchange insights to build collective knowledge  
- **Specialization**: Different agents focus on specific aspects of a problem  
- **Redundancy**: Multiple agents working on similar tasks improve reliability  
- **Emergent Intelligence**: The system exhibits capabilities beyond those of its individual components  

The **Strands Agents SDK** supports different multi-agent patterns such as:

- [**Swarm**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/swarm/)  
- [**Graph**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/graph/)  
- [**Workflow**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/workflow/)  
- [**Agent as a Tool**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/agents-as-tools/)


## Multi-Agent Collaboration

Below code is an implementation of the hierarchical topology pattern that implements a 2-level Multi agent structure with specialized roles. This hierarchical approach demonstrates one of the key topology patterns for agent graphs, showing how information flows through a tree-like structure with clear parent-child relationships.

In this lab we will be creating Strands agents similar to how we created in **2_bedrock-multi-agent** section.

1. **Supervisor agent** - Responsible for routing or coordinating tasks amongst the subagents.

2. **Collaborator agent 1** – Responsible for handling general mortgages.

3. **Collaborator agent 2** – Responsible for handling existing questions.

The below diagram depicts the graph

![Hierarchical Mortgage Agent](../../images/strands-multi-agent.drawio.png)

Install Strands Agents SDK and tools

In [1]:
!pip install strands-agents strands-agents-tools

In [2]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent
import sys
import pickle

from strands import Agent, tool
from strands_tools import retrieve, calculator


In [3]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)
boto3.set_stream_logger(name='botocore.credentials', level=logging.DEBUG)

In [4]:
%store -r kb_id
print("KnowledgeBase ID:",kb_id)

KnowledgeBase ID: OJOFGNEAFI


In [13]:
# Initialize knowledge base helper
# Set knowledge base ID as environment variable so that retrieve tool can use it
os.environ['KNOWLEDGE_BASE_ID'] = 'E726XJ2OEH'


In [14]:
value = os.getenv('KNOWLEDGE_BASE_ID')
print(value)

E726XJ2OEH


In [15]:
!echo $KNOWLEDGE_BASE_ID

E726XJ2OEH


# 1. Create the agent for general mortgage questions.

We will be creating an agent to answer general mortage questions from the Bedrock Knowledge Base created in **01_create_knowledgebase.ipynb** notebook in the **2_bedrock-multi-agent** section.





In [9]:


@tool
def answer_general_mortgage_questions(query):
    # Create the General Mortgage Agent
    general_mortgage_agent = Agent(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        tools=[
           retrieve
        ],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first.
        
        IMPORTANT: Always use the retrieve tool to search the knowledge base before answering any mortgage-related questions.
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinancing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access a knowledge base of mortgage information using the retrieve tool
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - ALWAYS call the retrieve tool first to search for relevant information
        - Provide clear explanations based on retrieved information
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )
    return str(general_mortgage_agent(query))



In [10]:
#alternative approach to pass kb id to the retrieve tool
'''
@tool
def answer_general_mortgage_questions(query):
    # Create the General Mortgage Agent
    general_mortgage_agent = Agent(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        tools=[retrieve],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first. Respond to the greeting by another greeting    
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinincing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access the bedrock knowledge base of mortgage information
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - Provide clear explanations
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        - Use the knowledge base for detailed information to answer general questions about mortgages, like how to refinance, or the difference between 15-year and 30-year mortgages
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )
    return str(general_mortgage_agent(query, tool_params={
                     "retrieve": {
                         "knowledge_base_id": kb_id,
                         "max_results": 5
                     }
                 }))

'''

'\n@tool\ndef answer_general_mortgage_questions(query):\n    # Create the General Mortgage Agent\n    general_mortgage_agent = Agent(\n        model="anthropic.claude-3-sonnet-20240229-v1:0",\n        tools=[retrieve],\n        system_prompt="""\n        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first. Respond to the greeting by another greeting    \n        \n        You can:\n        1. Provide general information about mortgages\n        2. Handle conversations about general mortgage questions, like high level concepts of refinincing or tradeoffs of 15-year vs 30-year terms.\n        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.\n        4. Access the bedrock knowledge base of mortgage information\n        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"\n        \n        

In [11]:
    '''
    #test connection with the knowledge base
    kb_id = 'E726XJ2OEH'
    os.environ['BEDROCK_KNOWLEDGE_BASE_ID'] = kb_id
    os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
    
    # Verify the knowledge base is accessible
    try:
        bedrock_client = boto3.client('bedrock-agent-runtime', region_name=os.environ['AWS_DEFAULT_REGION'])
        # Test connection (optional)
        print(f"Using Knowledge Base ID: {kb_id}")
    except Exception as e:
        print(f"AWS connection error: {e}")
    '''

'\n#test connection with the knowledge base\nkb_id = \'E726XJ2OEH\'\nos.environ[\'BEDROCK_KNOWLEDGE_BASE_ID\'] = kb_id\nos.environ[\'AWS_DEFAULT_REGION\'] = \'us-east-1\'\n\n# Verify the knowledge base is accessible\ntry:\n    bedrock_client = boto3.client(\'bedrock-agent-runtime\', region_name=os.environ[\'AWS_DEFAULT_REGION\'])\n    # Test connection (optional)\n    print(f"Using Knowledge Base ID: {kb_id}")\nexcept Exception as e:\n    print(f"AWS connection error: {e}")\n'

Test the General agent and confirm that it consults the KB to answer the questions

In [12]:
print(answer_general_mortgage_questions("What is the benefit of refinancing, if any?"))

2025-06-23 01:29:29,961 botocore.credentials [DEBUG] Looking for credentials via: env
[2025-06-23 01:29:29,961] p530 {credentials.py:2166} DEBUG - Looking for credentials via: env
2025-06-23 01:29:29,963 botocore.credentials [DEBUG] Looking for credentials via: assume-role
[2025-06-23 01:29:29,963] p530 {credentials.py:2166} DEBUG - Looking for credentials via: assume-role
2025-06-23 01:29:29,965 botocore.credentials [DEBUG] Looking for credentials via: assume-role-with-web-identity
[2025-06-23 01:29:29,965] p530 {credentials.py:2166} DEBUG - Looking for credentials via: assume-role-with-web-identity
2025-06-23 01:29:29,971 botocore.credentials [DEBUG] Looking for credentials via: sso
[2025-06-23 01:29:29,971] p530 {credentials.py:2166} DEBUG - Looking for credentials via: sso
2025-06-23 01:29:29,975 botocore.credentials [DEBUG] Looking for credentials via: shared-credentials-file
[2025-06-23 01:29:29,975] p530 {credentials.py:2166} DEBUG - Looking for credentials via: shared-credentia

Hello, let me first search the knowledge base for information on the benefits of refinancing a mortgage.
Tool #1: retrieve


2025-06-23 01:29:32,155 botocore.credentials [DEBUG] Looking for credentials via: env
[2025-06-23 01:29:32,155] p530 {credentials.py:2166} DEBUG - Looking for credentials via: env
2025-06-23 01:29:32,156 botocore.credentials [DEBUG] Looking for credentials via: assume-role
[2025-06-23 01:29:32,156] p530 {credentials.py:2166} DEBUG - Looking for credentials via: assume-role
2025-06-23 01:29:32,162 botocore.credentials [DEBUG] Looking for credentials via: assume-role-with-web-identity
[2025-06-23 01:29:32,162] p530 {credentials.py:2166} DEBUG - Looking for credentials via: assume-role-with-web-identity
2025-06-23 01:29:32,165 botocore.credentials [DEBUG] Looking for credentials via: sso
[2025-06-23 01:29:32,165] p530 {credentials.py:2166} DEBUG - Looking for credentials via: sso
2025-06-23 01:29:32,166 botocore.credentials [DEBUG] Looking for credentials via: shared-credentials-file
[2025-06-23 01:29:32,166] p530 {credentials.py:2166} DEBUG - Looking for credentials via: shared-credentia



Unfortunately I could not find any relevant information in the knowledge base about the benefits of refinancing a mortgage. However, based on general mortgage knowledge, some potential benefits of refinancing include:

- Getting a lower interest rate - This can reduce your monthly payments and total interest paid over the life of the loan.

- Shortening the loan term - For example, refinancing from a 30-year to a 15-year mortgage to pay off your home faster and save on total interest.

- Converting from an adjustable-rate to a fixed-rate mortgage - This can provide payment stability if interest rates rise.

- Cashing out home equity - With a cash-out refinance, you take out a new, larger mortgage and receive the difference in cash to use as you need.

However, refinancing also has costs like lender fees, closing costs, etc. The savings from the new terms need to be weighed against these upfront costs to determine if refinancing makes financial sense.

I would recommend speaking to a 

## 2. Create the agent for existing mortgage questions.

Create the Agent for managing existing mortgages



#### Create the Lambda function code
Here we create a source code file for a new Lambda function to implement the action group for our Existing Mortgage agent. Notice the **TODO** section in the code below. In this example we have hardcoded the response back from the agent but in your environment this is where your business logic would reside. 

In [ ]:
@tool
def get_mortgage_details(customer_id):
    # TODO: Implement real business logic to retrieve mortgage status
    return {
        "account_number": customer_id,
        "outstanding_principal": 150000.0,
        "interest_rate": 4.5,
        "maturity_date": "2030-06-30",
        "payments_remaining": 72,
        "last_payment_date": "2024-06-01",
        "next_payment_due": "2024-07-01",
        "next_payment_amount": 1250.0
    }


In [ ]:
@tool
def answer_existing_mortgage_questions(query):
    # Create the Existing Mortgage Agent
    existing_mortgage_agent = Agent(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        tools=[
           get_mortgage_details
        ],
        system_prompt="""
        You are an Existing Mortgage Assistant that helps customers with their current mortgages.

        You can:
        1. Provide information about a customer's existing mortgage
        2. Check mortgage status including balance and payment information
        3. Evaluate refinancing eligibility
        4. Calculate payoff timelines with extra payments
        5. Answer questions about mortgage terms and conditions

        When helping users:
        - Always verify the customer ID before providing information
        - Provide clear explanations of mortgage details
        - Format financial data in a readable way
        - Explain payment schedules and upcoming due dates
        - Offer guidance on refinancing options when appropriate
        - Use the knowledge base for detailed information when needed

        Remember that you're dealing with sensitive financial information, so maintain a professional tone
        and ensure accuracy in all responses.
        """
    )
    return str(existing_mortgage_agent(query))



Now we create the agent itself, giving it a name, 
a brief description, and most importantly, a set of instructions.

also briefly explain action group

we add an action group to the new agent

Test the agent

In [ ]:


print(answer_existing_mortgage_questions("I'm customer 98991. when's my next payment due?"))

## 3. Create the Supervisor agent



Create the supervisor agent

In [ ]:
SUPERVISOR_SYSTEM_PROMPT = """

Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
customer questions related to general mortgages questions and quries about the exiting mortgage

For general questions, use the answer_general_mortgage_questions tool.
For question on existing mortgage, use the answer_existing_mortgage_questions tool.
If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
Syntesise the details from te response of the tools used into a comprehensive answer provided back to the customer

"""

# Create the coordinator agent with all tools
supervisor = Agent(
    system_prompt=SUPERVISOR_SYSTEM_PROMPT,
    tools=[answer_general_mortgage_questions, answer_existing_mortgage_questions, calculator],
    callback_handler=None
)

# Process a complex task through the hierarchical agent graph
def answer_customer_query(query):
    """Process the customer query through the multi-level hierarchical agent graph"""
    return supervisor(f"Provide a comprehensive answer for this query: {query}")

In [ ]:
#test the supervisor agent
print("\n\nInvoking supervisor agent...\n\n")

requests = ["I am customer: 3345, when’s my next payment due?",
            "what’s my balance after that payment, and what rate am I paying?",
            "why do so many people choose a 30-year mortgage??",
            "did you receive my employment verification doc yet? i sent it last week",
            "i’m getting ready to lock in on a rate. what have the rates looked like in last couple weeks?",
            # "great. if i use the highest of those rates for $500K for 15 years, what’s my payment?"
            ]

for request in requests:
    print(f"\n\nRequest: {request}\n\n")
    result = answer_customer_query(request)
    time.sleep(10)
    print(result)